<a href="https://colab.research.google.com/github/pedrosof/CardioAI--Classificador/blob/main/diagnostico_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CardioIA – Parte 1: Extração de sintomas e sugestão de diagnósticos

Este notebook:
1. Lê um arquivo **.txt** com 10 frases de pacientes.
2. Lê o **mapa de conhecimento** (CSV) com pares de sintomas → doença associada.
3. Normaliza texto (minúsculas, sem acentos, sem pontuação).
4. Extrai sintomas por **casamento de expressões** (substrings/regex simples).
5. Gera um ranking de diagnósticos sugeridos por frase e salva a saída em CSV.

> **Atenção:** Tratamento **didático**. Não é ferramenta clínica.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import os
import re
import json
import unicodedata
import pandas as pd
from pathlib import Path

# Caminhos base no Google Drive
BASE_DIR = Path("/content/drive/MyDrive/CardioAI/Classificador")

# Subpastas dentro do diretório do projeto
DATA_DIR = BASE_DIR / "data"
REPORTS_DIR = BASE_DIR / "reports"

# Arquivos de entrada e saída
TXT_FRASES = DATA_DIR / "sintomas.txt"
CSV_MAPA = DATA_DIR / "mapa_sintomas.csv"
CSV_SAIDA = REPORTS_DIR / "diagnosticos_preditos.csv"

# Criação das pastas (somente reports, já que data deve existir)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"DATA_DIR   = {DATA_DIR}")
print(f"REPORTS_DIR = {REPORTS_DIR}")
print(f"TXT_FRASES = {TXT_FRASES}")
print(f"CSV_MAPA   = {CSV_MAPA}")
print(f"CSV_SAIDA  = {CSV_SAIDA}")

DATA_DIR   = /content/drive/MyDrive/CardioAI/Classificador/data
REPORTS_DIR = /content/drive/MyDrive/CardioAI/Classificador/reports
TXT_FRASES = /content/drive/MyDrive/CardioAI/Classificador/data/sintomas.txt
CSV_MAPA   = /content/drive/MyDrive/CardioAI/Classificador/data/mapa_sintomas.csv
CSV_SAIDA  = /content/drive/MyDrive/CardioAI/Classificador/reports/diagnosticos_preditos.csv


In [7]:
def remover_acentos(txt: str) -> str:
    nfkd = unicodedata.normalize("NFKD", txt)
    return "".join([c for c in nfkd if not unicodedata.combining(c)])

def normalizar(txt: str) -> str:
    # minúsculas, remove acentos e pontuação simples
    txt = txt.lower()
    txt = remover_acentos(txt)
    # troca pontuação por espaço
    txt = re.sub(r'[\.,;:!?\-/()\[\]{}"]+', " ", txt)
    # normaliza múltiplos espaços
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

def contem_expressao(frase_norm: str, expressao_norm: str) -> bool:
    # busca por substring com margens de palavra aproximadas quando possível
    # se a expressao tiver espaços, busca substring simples
    if " " in expressao_norm:
        return expressao_norm in frase_norm
    # caso seja uma palavra única, usar limites de palavra
    return re.search(rf"\b{re.escape(expressao_norm)}\b", frase_norm) is not None

In [8]:
# Carregando o mapa e as frases
mapa = pd.read_csv(CSV_MAPA)
with open(TXT_FRASES, "r", encoding="utf-8") as f:
    linhas = [ln.strip() for ln in f.readlines() if ln.strip()]

print("Exemplo do mapa (5 primeiras linhas):")
display(mapa.head())

print("\nFrases (detectadas):")
for i, ln in enumerate(linhas, 1):
    print(f"{i}. {ln}")

Exemplo do mapa (5 primeiras linhas):


,sintoma_1,sintoma_2,doenca_associada
0,dor no peito,aperto no torax,Infarto agudo do miocardio
1,falta de ar,dificuldade para respirar,Angina
2,cansaco constante,fadiga,Insuficiencia cardiaca
3,palpitacoes,batimentos acelerados,Arritmia
4,tontura,desmaio,Sincopa cardiogenica



Frases (detectadas):
1. Há dois dias estou com uma dor no peito que piora quando faço esforço físico e melhora quando descanso.
2. Sinto cansaço constante há uma semana, mesmo depois de dormir bem e evitar atividades pesadas.
3. Tenho sentido falta de ar ao subir escadas e um leve aperto no peito que vem e vai durante o dia.
4. Há três dias sinto tontura e palpitações repentinas, principalmente quando fico em pé por muito tempo.
5. Desde ontem estou com inchaço nas pernas e tornozelos, e percebi que estou urinando menos que o normal.
6. Tenho sentido dores de cabeça e uma sensação de pressão no peito toda vez que fico nervoso ou ansioso.
7. Sinto uma dor leve nas costas e um desconforto no peito quando respiro fundo há cerca de quatro dias.
8. Estou com uma tosse seca persistente há mais de uma semana e às vezes sinto falta de ar ao deitar.
9. Há cerca de dois dias senti uma dor forte no peito que irradiou para o braço esquerdo e veio acompanhada de suor frio.
10. Tenho percebido bati

In [9]:
# Prepara expressões normalizadas do mapa para facilitar matching
mapa_proc = []
for _, row in mapa.iterrows():
    s1 = normalizar(str(row["sintoma_1"]))
    s2 = normalizar(str(row["sintoma_2"]))
    doenca = str(row["doenca_associada"]).strip()
    if s1:
        mapa_proc.append((s1, doenca))
    if s2:
        mapa_proc.append((s2, doenca))

# Dicionário: expressao_norm -> set(doencas)
from collections import defaultdict
expressao_to_doencas = defaultdict(set)
for expr, dz in mapa_proc:
    expressao_to_doencas[expr].add(dz)

len(expressao_to_doencas), list(list(expressao_to_doencas.items())[:5])

(92,
 [('dor no peito', {'Infarto agudo do miocardio'}),
  ('aperto no torax', {'Infarto agudo do miocardio'}),
  ('falta de ar', {'Angina'}),
  ('dificuldade para respirar', {'Angina'}),
  ('cansaco constante', {'Insuficiencia cardiaca'})])

In [10]:
def sugerir_diagnosticos(frase: str, expressao_map: dict, top_k=3):
    frase_norm = normalizar(frase)
    matches = []
    score = {}

    # Verifica cada expressao do mapa na frase
    for expr, doencas in expressao_map.items():
        if contem_expressao(frase_norm, expr):
            matches.append(expr)
            for dz in doencas:
                score[dz] = score.get(dz, 0) + 1  # soma 1 por match de expressão

    # Ordena por pontuação decrescente e alfabética como tie-break
    rank = sorted(score.items(), key=lambda x: (-x[1], x[0]))
    sugestoes = [dz for dz, sc in rank[:top_k]]
    return {
        "frase": frase,
        "frase_norm": frase_norm,
        "matches": matches,
        "diagnosticos_rankeados": rank,
        "sugestoes_topk": sugestoes
    }

resultados = [sugerir_diagnosticos(fr, expressao_to_doencas, top_k=3) for fr in linhas]

# Mostra alguns resultados
for i, r in enumerate(resultados, 1):
    print("="*80)
    print(f"[{i}] {r['frase']}")
    print(f"Matches: {', '.join(r['matches']) if r['matches'] else '(nenhum)'}")
    if r["diagnosticos_rankeados"]:
        print("Diagnósticos sugeridos (doença:score):")
        for dz, sc in r["diagnosticos_rankeados"]:
            print(f" - {dz}: {sc}")
        print(f"TOP: {r['sugestoes_topk']}")
    else:
        print("Nenhum diagnóstico sugerido com base no mapa.")

[1] Há dois dias estou com uma dor no peito que piora quando faço esforço físico e melhora quando descanso.
Matches: dor no peito
Diagnósticos sugeridos (doença:score):
 - Infarto agudo do miocardio: 1
TOP: ['Infarto agudo do miocardio']
[2] Sinto cansaço constante há uma semana, mesmo depois de dormir bem e evitar atividades pesadas.
Matches: cansaco constante, cansaco
Diagnósticos sugeridos (doença:score):
 - Insuficiencia cardiaca: 1
 - Insuficiencia cardiaca investigar: 1
TOP: ['Insuficiencia cardiaca', 'Insuficiencia cardiaca investigar']
[3] Tenho sentido falta de ar ao subir escadas e um leve aperto no peito que vem e vai durante o dia.
Matches: falta de ar
Diagnósticos sugeridos (doença:score):
 - Angina: 1
TOP: ['Angina']
[4] Há três dias sinto tontura e palpitações repentinas, principalmente quando fico em pé por muito tempo.
Matches: palpitacoes, tontura
Diagnósticos sugeridos (doença:score):
 - Arritmia: 1
 - Arritmia investigar: 1
 - Sincopa cardiogenica: 1
TOP: ['Arritmia

In [11]:
# Exporta uma tabela consolidada com resultados
linhas_out = []
for r in resultados:
    linhas_out.append({
        "frase": r["frase"],
        "matches_encontrados": "; ".join(r["matches"]),
        "doencas_sugeridas_top3": "; ".join(r["sugestoes_topk"]) if r["sugestoes_topk"] else ""
    })

df_out = pd.DataFrame(linhas_out, columns=["frase", "matches_encontrados", "doencas_sugeridas_top3"])
df_out.to_csv(CSV_SAIDA, index=False, encoding="utf-8")
print(f"Arquivo gerado: {CSV_SAIDA.resolve()}")
display(df_out)

Arquivo gerado: /content/drive/MyDrive/CardioAI/Classificador/reports/diagnosticos_preditos.csv


,frase,matches_encontrados,doencas_sugeridas_top3
0,Há dois dias estou com uma dor no peito que pi...,dor no peito,Infarto agudo do miocardio
1,"Sinto cansaço constante há uma semana, mesmo d...",cansaco constante; cansaco,Insuficiencia cardiaca; Insuficiencia cardiaca...
2,Tenho sentido falta de ar ao subir escadas e u...,falta de ar,Angina
3,Há três dias sinto tontura e palpitações repen...,palpitacoes; tontura,Arritmia; Arritmia investigar; Sincopa cardiog...
4,Desde ontem estou com inchaço nas pernas e tor...,inchaco nas pernas,Insuficiencia cardiaca descompensada
5,Tenho sentido dores de cabeça e uma sensação d...,pressao no peito,Angina
6,Sinto uma dor leve nas costas e um desconforto...,,
7,Estou com uma tosse seca persistente há mais d...,falta de ar; falta de ar ao deitar; tosse seca...,Insuficiencia cardiaca; Angina; Pneumonia atipica
8,Há cerca de dois dias senti uma dor forte no p...,suor frio,Choque cardiogenico; Sindrome coronariana aguda
9,Tenho percebido batimentos acelerados e irregu...,batimentos acelerados,Arritmia


## Observações e próximos passos

- Ajuste/expansão do **mapa de sintomas** para melhorar cobertura e precisão.
- Adaptação de **regras de pontuação** (e.g., peso maior para certas expressões como “dor no peito + suor frio”).
- Uso **didático**, não clínico.